In [19]:
## Initialize
import spacy
from spacy.language import Language

import requests
import json
import numpy as np
import re

apiKey = "hTLGygty2Sj5IB368rcArA63Xu29hW2r"
archiveUrl = f'https://api.nytimes.com/svc/archive/v1/#1/#2.json?api-key={apiKey}'

def parseDoc(doc) -> list[str]:
  headline = doc["headline"]["print_headline"] or doc["headline"]["main"]
  leadPara = doc["lead_paragraph"]
  return str(headline + (" " if leadPara else "") + leadPara)

# Disabling components not needed (optional, but useful if run on a large dataset)
nlp = spacy.load("en_core_web_lg", disable=["tok2vec", "parser", "senter", "lemmatizer", "tagger", "attribute_ruler"])
nlp.add_pipe("merge_noun_chunks")
nlp.add_pipe("merge_entities")

# Load the larger model for similarity calculation
nlp_lg = nlp

# Example text
text = "German Chancellor Angela Merkel died in 1936 in New York. She got shot by a mysterious terrorist, terror, terrorism"

In [20]:
## Fetch

print(archiveUrl.replace("#1", "1963").replace("#2", "10"))
res = requests.get(archiveUrl.replace("#1", "1963").replace("#2", "10"))
obj = json.loads(res.text)
#words = " ".join(map(lambda doc: parseDoc(doc), list(obj["response"]["docs"])))
words = "HIDE KILLED PAKISTAN STATEMENT MALAYSIA CONGRESS PLOT ALABAMA SUITS STATEMENT MALAYSIA CONGRESS Killed"

words


https://api.nytimes.com/svc/archive/v1/1963/10.json?api-key=hTLGygty2Sj5IB368rcArA63Xu29hW2r


'HIDE KILLED PAKISTAN STATEMENT MALAYSIA CONGRESS PLOT ALABAMA SUITS STATEMENT MALAYSIA CONGRESS Killed'

In [21]:
## Search

def checkMatches(token, relevant_words):
    token_lg = nlp_lg(token.text)
    for comp in relevant_words.keys():
        comp_lg = nlp_lg(comp)
        similarity = token_lg.similarity(comp_lg)
        if(similarity >= 0.8):
            return (comp, similarity)
    return False

# Task 1: Extracting relevant words using the transformer-based model
wordCount = {}
matches: dict[str, dict] = {}
doc_trf = nlp(words)
for token in doc_trf:
    if token.text in matches.keys():
        matches[token.text]["amount"] = matches[token.text]["amount"] + 1
    elif not token.is_stop and not token.is_punct:
        matched = checkMatches(token, wordCount)
        if(not matched):
            wordCount[token.text] = {}
            wordCount[token.text]["amount"] = 1
        else:
            if token.text in matches.keys():
                matches[token.text]["amount"] = matches[token.text]["amount"] + 1
            else:
                matches[token.text] = {
                    "match": matched[0],
                    "similarity": matched[1],
                    "amount": 1
                }




In [25]:
for m in matches.items():
    if "duplicates" not in wordCount[m[1]["match"]]:
        wordCount[m[1]["match"]]["duplicates"] = {}
    wordCount[m[1]["match"]]["duplicates"][m[0]] = {
        "amount": m[1]["amount"],
        "similarity": m[1]["similarity"]
    }
    wordCount[m[1]["match"]]["amount"] = wordCount[m[1]["match"]]["amount"] + m[1]["amount"]

wordCount

{'HIDE': {'amount': 7,
  'duplicates': {'KILLED': {'amount': 1, 'similarity': 0.8057828600326064},
   'PLOT': {'amount': 1, 'similarity': 0.8421579273476232},
   'SUITS': {'amount': 1, 'similarity': 0.8514908262080949}}},
 'PAKISTAN': {'amount': 11,
  'duplicates': {'MALAYSIA': {'amount': 2, 'similarity': 0.819214576299081},
   'CONGRESS': {'amount': 2, 'similarity': 0.8360562373005461},
   'ALABAMA': {'amount': 1, 'similarity': 0.8322161143134507}}},
 'STATEMENT': {'amount': 3,
  'duplicates': {'STATEMENT': {'amount': 1, 'similarity': 1.0}}},
 'Killed': {'amount': 1}}

In [23]:
print(wordCount)
print("\n--------------------\n")
print(matches)

{'HIDE': {'amount': 4, 'duplicates': {'KILLED': 1, 'PLOT': 1, 'SUITS': 1}}, 'PAKISTAN': {'amount': 6, 'duplicates': {'MALAYSIA': 2, 'CONGRESS': 2, 'ALABAMA': 1}}, 'STATEMENT': {'amount': 2, 'duplicates': {'STATEMENT': 1}}, 'Killed': {'amount': 1}}

--------------------

{'KILLED': {'match': 'HIDE', 'similarity': 0.8057828600326064, 'amount': 1}, 'MALAYSIA': {'match': 'PAKISTAN', 'similarity': 0.819214576299081, 'amount': 2}, 'CONGRESS': {'match': 'PAKISTAN', 'similarity': 0.8360562373005461, 'amount': 2}, 'PLOT': {'match': 'HIDE', 'similarity': 0.8421579273476232, 'amount': 1}, 'ALABAMA': {'match': 'PAKISTAN', 'similarity': 0.8322161143134507, 'amount': 1}, 'SUITS': {'match': 'HIDE', 'similarity': 0.8514908262080949, 'amount': 1}, 'STATEMENT': {'match': 'STATEMENT', 'similarity': 1.0, 'amount': 1}}


In [24]:
import spacy
taggers = [{
    "name": "Taggers",
    "values": [
        "$", "''", ",", "-LRB-", "-RRB-", ".", ":", "ADD", "AFX", "CC", "CD", "DT", "EX", "FW", "HYPH", "IN", "JJ", "JJR", "JJS", "LS", "MD", "NFP", "NN", "NNP", "NNPS", "NNS", "PDT", "POS", "PRP", "PRP$", "RB", "RBR", "RBS", "RP", "SYM", "TO", "UH", "VB", "VBD", "VBG", "VBN", "VBP", "VBZ", "WDT", "WP", "WP$", "WRB", "XX", "_SP", "``"
    ]
}]

for tagger in taggers:
    print(tagger["name"])
    print(tagger["values"])

for tagger in taggers:

    for tag in tagger["values"]:
        print(f"{tag["name"]}: {spacy.explain(tag)}")

SyntaxError: f-string: unmatched '[' (3698956332.py, line 16)